In [1]:
import statsmodels.api as sm
import pylab as pl
import numpy as np
import MySQLdb as mdb
import getpass

In [2]:
# Connect to the MySQL database containing the results of import_game_pbp_data.py
rootpass = getpass.getpass('Enter the database root password: ')
con = mdb.connect('localhost', 'root', rootpass, 'NFL_Offensive_Plays_2015')
cur = con.cursor()

Enter the database root password: ········


In [79]:
# A list of the teams. Teams that switched offensive coordinators during the 2015 season: Detroit Lions (Oct 26, after Week 7), Indianapolis Colts (Nov 3, after Week 8), St. Louis Rams (Dec. 7, after Week 13)
teams = ['Baltimore Ravens', 'Cincinnati Bengals', 'Cleveland Browns', 'Pittsburgh Steelers', 'Houston Texans', 'Indianapolis Colts', 'Jacksonville Jaguars', 'Tennessee Titans', 'Buffalo Bills', 'Miami Dolphins', 'New England Patriots', 'New York Jets', 'Denver Broncos', 'Kansas City Chiefs', 'Oakland Raiders', 'San Diego Chargers', 'Chicago Bears', 'Detroit Lions', 'Green Bay Packers', 'Minnesota Vikings', 'Atlanta Falcons', 'Carolina Panthers', 'New Orleans Saints', 'Tampa Bay Buccaneers', 'Dallas Cowboys', 'New York Giants', 'Philadelphia Eagles', 'Washington Redskins', 'Arizona Cardinals', 'St. Louis Rams', 'San Francisco 49ers', 'Seattle Seahawks']
#Below, I initialize the lists of parameters I'll use for later analysis.
logitList = []
fitResult = []
#summaryList = []
predIsTrue = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
fracCorrect = []
numPass = []
fracPass = []
fracTruePass = []
fracFalsePass = []
fracTrueRun = []
fracFalseRun = []
fracUncertain = []
fracUncertainOrSureRun = []
F1Pass = []
F1Run = []
# A list of the total yards for each team, in the order corresponding to that of teams.
totYds = [5749,5728,5311,6327,5564,5142,5581,4988,5775,5307,5991,5925,5688,5299,5336,5949,5514,5547,5353,5139,5990,5871,6461,6014,5361,5956,5830,5661,6533,4761,4860,6058]
pts = [328,419,278,423,339,333,376,299,379,310,465,387,355,405,359,320,335,358,368,365,339,500,408,342,275,420,377,388,489,280,238,423]
# For each team, I will perform logistic regression on the offensive plays data from the first half of the season to try to 
# predict whether plays from the second half of the season will be a run or a pass.
j = 0
while j < len(teams):
    #The first step is to get the data from the first half of the season from the MySQL database.
    cur.execute("SELECT `Time Remaining`, Down, `To Go`, `Field Position`, `Score Differential` FROM `" + teams[j] + "` WHERE Week < 9")
    firstHalfData = cur.fetchall()
    cur.execute("SELECT IsPass FROM `" + teams[j] + "` WHERE Week < 9")
    firstHalfOut = cur.fetchall()
    
    # I call the inputs from the training set  firstHalfIn. Each row in firstHalfIn includes the bias term, the down,
    # the distance, the field position (a number between 0 and 50; it doesn't differentiate between sides of the field), and 
    # (score differential) divided by (time remaining in seconds plus .01). I expect the probability of a pass to change 
    # monotonically with each non-bias input: The higher the down, the higher the distance, the farther from either goalline, and
    # the lower the ratio between score differential and time remaining, the higher I expect the probability of a pass to be. The 
    # .01 is in the denominator of the last input to avoid singularities on the rare occasions that a defensive penalty allows for
    # an offensive play with no time remaining on the clock.
    firstHalfIn = []
    for play in firstHalfData:
        firstHalfIn.append([1, int(play[1]), int(play[2]), int(min(play[3], 100-play[3])), float(play[4])/(float(play[0])+.01)])

    # I save the results of the fit result in a list.
    logitList.append(sm.Logit(firstHalfOut, firstHalfIn))
    fitResult.append(logitList[j].fit())
    #summaryList.append(result[j].summary())

    #Now, I get the data from the second half of the season (weeks 9-17)
    cur.execute("SELECT `Time Remaining`, Down, `To Go`, `Field Position`, `Score Differential` FROM `" + teams[j] + "` WHERE Week > 8")
    secondHalfData = cur.fetchall()
    cur.execute("SELECT IsPass FROM `" + teams[j] + "` WHERE Week > 8")
    secondHalfOut = cur.fetchall()
    secondHalfIn = []
    for play in secondHalfData:
        secondHalfIn.append([1, int(play[1]), int(play[2]), int(min(play[3], 100-play[3])), float(play[4])/(float(play[0])+.01)])

    # I now need to use the results of the logistic regression to make predictions about the second half of the season. Dotting
    # the second half data into the fitting parameters gives a list of sigmoid funtion arguments. Applying the sigmoid function 
    # to them gives a list of pass probabilites. Rounding these probabilities gives the prediction of a run or pass.
    secondHalfPred = []
    sigmoids = 1/(1+np.exp(-np.dot(secondHalfIn, fitResult[j].params)))
    for sigmoid in sigmoids:
        secondHalfPred.append(round(sigmoid))

    # The following while loop fills in the list predIsTrue[j], with ones indicating correct predictions, and zeros indicating
    # incorrect predictions. It also counts the number of true pass, true run, false pass, and false run predictions there are.
    i = 0
    tempTruePass = 0
    tempTrueRun = 0
    tempFalsePass = 0
    tempFalseRun = 0
    tempUncertain = 0
    tempUncertainOrSureRun = 0
    while i < len(secondHalfPred):
        if sigmoids[i] < .6:
            tempUncertainOrSureRun += 1
            if sigmoids[i] > .4:
                tempUncertain += 1
        if secondHalfPred[i] == 1 and secondHalfOut[i][0] == 1:
            predIsTrue[j].append(1)
            tempTruePass += 1
        elif secondHalfPred[i] == 0 and secondHalfOut[i][0] == 0:
            predIsTrue[j].append(1)
            tempTrueRun += 1
        elif secondHalfPred[i] == 1 and secondHalfOut[i][0] == 0:
            predIsTrue[j].append(0)
            tempFalsePass += 1
        elif secondHalfPred[i] == 0 and secondHalfOut[i][0] == 1:
            predIsTrue[j].append(0)
            tempFalseRun += 1
        i += 1
    
    # We now calculate the fraction of second half plays that were correct predictions, correct pass predictions, correct run 
    # predictions, false pass predictions, and false run predictions.
    fracCorrect.append(float(sum(predIsTrue[j]))/float(len(predIsTrue[j])))
    fracTruePass.append(float(tempTruePass)/float(len(predIsTrue[j])))
    fracTrueRun.append(float(tempTrueRun)/float(len(predIsTrue[j])))
    fracFalsePass.append(float(tempFalsePass)/float(len(predIsTrue[j])))
    fracFalseRun.append(float(tempFalseRun)/float(len(predIsTrue[j])))
    fracUncertain.append(float(tempUncertain)/float(len(predIsTrue[j])))
    fracUncertainOrSureRun.append(float(tempUncertainOrSureRun)/float(len(predIsTrue[j])))
    
    # The next few lines find the total number of passing plays, as well as the fraction of pass plays in the second half of the
    # season.
    tempNumPass = 0
    for play in secondHalfOut:
        tempNumPass += play[0]
    numPass.append(tempNumPass)
    fracPass.append(float(numPass[j])/float(len(secondHalfOut)))
    
    # The following two lines calculate the F1 score while treating 'pass' as the 'positive' value, and then the same with 'run'
    # as the 'positive' value.
    F1Pass.append(2.0*fracTruePass[j]/(2.0*fracTruePass[j] + fracFalsePass[j] + fracFalseRun[j]))
    F1Run.append(2.0*fracTrueRun[j]/(2.0*fracTrueRun[j] + fracFalseRun[j] + fracFalsePass[j]))
    j += 1

Optimization terminated successfully.
         Current function value: 0.591050
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.654667
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.573735
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.611577
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.575147
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.582194
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.606127
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.599642
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.601416
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.601603
 

In [44]:
fracCorrect

[0.6789667896678967,
 0.68,
 0.6535433070866141,
 0.55893536121673,
 0.6311787072243346,
 0.6360078277886497,
 0.6703296703296703,
 0.6610486891385767,
 0.6648550724637681,
 0.6338797814207651,
 0.6303972366148531,
 0.6766256590509666,
 0.6454388984509466,
 0.5803571428571429,
 0.6289752650176679,
 0.6679389312977099,
 0.6696588868940754,
 0.6257309941520468,
 0.5953125,
 0.7102803738317757,
 0.6750972762645915,
 0.6473594548551959,
 0.6844106463878327,
 0.6433566433566433,
 0.6819047619047619,
 0.6383763837638377,
 0.5958132045088567,
 0.5959780621572212,
 0.6400778210116731,
 0.7038461538461539,
 0.658008658008658,
 0.6227897838899804]

In [8]:
fracPass

[0.6568265682656826,
 0.5290909090909091,
 0.6259842519685039,
 0.6615969581749049,
 0.5437262357414449,
 0.6203522504892368,
 0.6703296703296703,
 0.5936329588014981,
 0.4927536231884058,
 0.6284153005464481,
 0.6044905008635578,
 0.6028119507908611,
 0.5972461273666093,
 0.5200892857142857,
 0.6183745583038869,
 0.6297709923664122,
 0.5224416517055656,
 0.6101364522417154,
 0.6140625,
 0.5158878504672897,
 0.6186770428015564,
 0.4991482112436116,
 0.6026615969581749,
 0.5804195804195804,
 0.5866666666666667,
 0.5940959409594095,
 0.5668276972624798,
 0.5850091407678245,
 0.5953307392996109,
 0.551923076923077,
 0.6385281385281385,
 0.49901768172888017]

In [63]:
fracUncertain

[0.48523985239852396,
 0.649090909090909,
 0.4015748031496063,
 0.3669201520912547,
 0.44106463878326996,
 0.48140900195694714,
 0.47802197802197804,
 0.41198501872659177,
 0.4963768115942029,
 0.21311475409836064,
 0.10535405872193437,
 0.4182776801405975,
 0.3270223752151463,
 0.5513392857142857,
 0.3568904593639576,
 0.4312977099236641,
 0.3087971274685817,
 0.4502923976608187,
 0.5078125,
 0.5626168224299065,
 0.3949416342412451,
 0.3577512776831346,
 0.4296577946768061,
 0.27972027972027974,
 0.3980952380952381,
 0.3929889298892989,
 0.5442834138486312,
 0.4954296160877514,
 0.37937743190661477,
 0.24807692307692308,
 0.4588744588744589,
 0.5913555992141454]

In [53]:
linfitCorrect = sm.OLS(totYds, sm.add_constant(fracCorrect))
linfitCorrectresult = linfitCorrect.fit()
xvalsCorrect = [.55, .72]
yvalsCorrect = [linfitCorrectresult.params[1]*xvalsCorrect[0] + linfitCorrectresult.params[0], linfitCorrectresult.params[1]*xvalsCorrect[1] + linfitCorrectresult.params[0]]
plot1 = pl.plot(fracCorrect,totYds,'gx', xvalsCorrect, yvalsCorrect)
pl.ylabel('Total Yards')
pl.xlabel('Fraction of Correct Predictions')

In [54]:
pl.show(plot1)

In [49]:
linfitF1Pass = sm.OLS(totYds, sm.add_constant(F1Pass))
linfitF1Passresult = linfitF1Pass.fit()
xvalsF1Pass = [.55, .80]
yvalsF1Pass = [linfitF1Passresult.params[1]*xvalsF1Pass[0] + linfitF1Passresult.params[0], linfitF1Passresult.params[1]*xvalsF1Pass[1] + linfitF1Passresult.params[0]]
plotF1Pass = pl.plot(F1Pass,totYds,'gx', xvalsF1Pass, yvalsF1Pass)

In [50]:
pl.show(plotF1Pass)

In [55]:
linfitF1Run = sm.OLS(totYds, sm.add_constant(F1Run))
linfitF1Runresult = linfitF1Run.fit()
xvalsF1Run = [0, .7]
yvalsF1Run = [linfitF1Runresult.params[1]*xvalsF1Run[0] + linfitF1Runresult.params[0], linfitF1Runresult.params[1]*xvalsF1Run[1] + linfitF1Runresult.params[0]]
plotF1Run = pl.plot(F1Run,totYds,'gx', xvalsF1Run, yvalsF1Run)

In [56]:
pl.show(plotF1Run)

In [ ]:
difference = []
j = 0
while j < len(fracCorrect):
    difference.append(fracCorrect[j]-fracPass[j])
    j += 1

In [ ]:
difference

In [60]:
linfitdiff = sm.OLS(totYds, sm.add_constant(difference))
linfitdiffresult = linfitdiff.fit()
xvalsdiff = [-.04, .14]
yvalsdiff = [linfitdiffresult.params[1]*xvalsdiff[0] + linfitdiffresult.params[0], linfitdiffresult.params[1]*xvalsdiff[1] + linfitdiffresult.params[0]]
plotdiff = pl.plot(difference, totYds, 'gx', xvalsdiff, yvalsdiff)

In [61]:
pl.show(plotdiff)

In [73]:
OLSUncertain = sm.OLS(pts, sm.add_constant(fracUncertain))
fitUncertain = OLSUncertain.fit()
xvalsUncertain = [.1, .7]
yvalsUncertain = [fitUncertain.params[1]*xvalsUncertain[0] + fitUncertain.params[0], fitUncertain.params[1]*xvalsUncertain[1] + fitUncertain.params[0]]
plotUncertain = pl.plot(fracUncertain, pts, 'gx', xvalsUncertain, yvalsUncertain)

In [74]:
pl.show(plotUncertain)

In [75]:
fitUncertain.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.030
Method:                 Least Squares   F-statistic:                   0.09067
Date:                Fri, 27 Jan 2017   Prob (F-statistic):              0.765
Time:                        21:37:00   Log-Likelihood:                -176.35
No. Observations:                  32   AIC:                             356.7
Df Residuals:                      30   BIC:                             359.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const        352.5868     42.647      8.267      0.000       265.489   439.685
x1            29.6101     98.334      0.301      0.765      -171.215   230.435
==============================================================================
Omnibus:                        0.879   Durbin-Watson:                   2.326
Prob(Omnibus):                  0.644   Jarque-Bera (JB):                0.446
Skew:                           0.289   Prob(JB):                        0.800
Kurtosis:                       3.028   Cond. No.                         10.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [86]:
OLSUncertainOrSureRun = sm.OLS(pts, sm.add_constant(fracUncertainOrSureRun))
fitUncertainOrSureRun = OLSUncertainOrSureRun.fit()
xvalsUncertainOrSureRun = [.1, .9]
yvalsUncertainOrSureRun = [fitUncertainOrSureRun.params[1]*xvalsUncertainOrSureRun[0] + fitUncertainOrSureRun.params[0], fitUncertainOrSureRun.params[1]*xvalsUncertainOrSureRun[1] + fitUncertainOrSureRun.params[0]]
plotUncertainOrSureRun = pl.plot(fracUncertainOrSureRun, pts, 'gx', xvalsUncertainOrSureRun, yvalsUncertainOrSureRun)

In [87]:
pl.show(plotUncertainOrSureRun)

In [85]:
fitUncertainOrSureRun.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.9173
Date:                Fri, 27 Jan 2017   Prob (F-statistic):              0.346
Time:                        21:43:02   Log-Likelihood:                -175.91
No. Observations:                  32   AIC:                             355.8
Df Residuals:                      30   BIC:                             358.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const        323.2772     44.878      7.203      0.000       231.623   414.931
x1            76.8888     80.282      0.958      0.346       -87.069   240.847
==============================================================================
Omnibus:                        0.833   Durbin-Watson:                   2.198
Prob(Omnibus):                  0.659   Jarque-Bera (JB):                0.376
Skew:                           0.264   Prob(JB):                        0.829
Kurtosis:                       3.061   Cond. No.                         9.67
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [88]:
dir(fitUncertainOrSureRun)

['HC0_se',
 'HC1_se',
 'HC2_se',
 'HC3_se',
 '_HCCM',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cache',
 '_data_attr',
 '_get_robustcov_results',
 '_is_nested',
 '_wexog_singular_values',
 'aic',
 'bic',
 'bse',
 'centered_tss',
 'compare_f_test',
 'compare_lm_test',
 'compare_lr_test',
 'condition_number',
 'conf_int',
 'conf_int_el',
 'cov_HC0',
 'cov_HC1',
 'cov_HC2',
 'cov_HC3',
 'cov_kwds',
 'cov_params',
 'cov_type',
 'df_model',
 'df_resid',
 'eigenvals',
 'el_test',
 'ess',
 'f_pvalue',
 'f_test',
 'fittedvalues',
 'fvalue',
 'get_influence',
 'get_robustcov_results',
 'initialize',
 'k_constant',
 'llf',
 'load',
 'model',
 'mse_model',
 'mse_resid',
 'mse_total',
 'nobs',
 'normalized_cov_params',
 'outlier_test',
 'params',
 'predict',
 'pvalues',


In [111]:
fitUncertainOrSureRun.conf_int_el(1,sig = .477)

(-0.11943372217607154, 153.2644912452145)

In [123]:
OLSUncertainOrSureRunYds = sm.OLS(totYds, sm.add_constant(fracUncertainOrSureRun))
fitUncertainOrSureRunYds = OLSUncertainOrSureRunYds.fit()
xvalsUncertainOrSureRunYds = [.1, .9]
yvalsUncertainOrSureRunYds = [fitUncertainOrSureRunYds.params[1]*xvalsUncertainOrSureRunYds[0] + fitUncertainOrSureRunYds.params[0], fitUncertainOrSureRunYds.params[1]*xvalsUncertainOrSureRunYds[1] + fitUncertainOrSureRunYds.params[0]]
plotUncertainOrSureRunYds = pl.plot(fracUncertainOrSureRun, totYds, 'gx', xvalsUncertainOrSureRunYds, yvalsUncertainOrSureRunYds)

In [124]:
pl.show(plotUncertainOrSureRunYds)

In [114]:
yvalsUncertainOrSureRunYds

[330.96611200578815, 392.47718507000644]

In [120]:
fitUncertainOrSureRunYds.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                 -0.020
Method:                 Least Squares   F-statistic:                    0.3894
Date:                Fri, 27 Jan 2017   Prob (F-statistic):              0.537
Time:                        22:35:33   Log-Likelihood:                -239.02
No. Observations:                  32   AIC:                             482.0
Df Residuals:                      30   BIC:                             485.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       5447.4263    322.511     16.891      0.000      4788.771  6106.082
x1           360.0099    576.936      0.624      0.537      -818.250  1538.270
==============================================================================
Omnibus:                        0.011   Durbin-Watson:                   2.061
Prob(Omnibus):                  0.994   Jarque-Bera (JB):                0.176
Skew:                          -0.031   Prob(JB):                        0.916
Kurtosis:                       2.642   Cond. No.                         9.67
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
linfitF1Passresult.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.020
Date:                Tue, 01 Nov 2016   Prob (F-statistic):              0.321
Time:                        00:58:28   Log-Likelihood:                -238.70
No. Observations:                  32   AIC:                             481.4
Df Residuals:                      30   BIC:                             484.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       6591.3655    942.445      6.994      0.000      4666.636  8516.095
x1         -1340.2919   1327.202     -1.010      0.321     -4050.800  1370.216
==============================================================================
Omnibus:                        0.034   Durbin-Watson:                   2.112
Prob(Omnibus):                  0.983   Jarque-Bera (JB):                0.236
Skew:                          -0.008   Prob(JB):                        0.889
Kurtosis:                       2.579   Cond. No.                         26.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
linfitF1Runresult.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.033
Method:                 Least Squares   F-statistic:                 0.0001313
Date:                Tue, 01 Nov 2016   Prob (F-statistic):              0.991
Time:                        01:00:58   Log-Likelihood:                -239.23
No. Observations:                  32   AIC:                             482.5
Df Residuals:                      30   BIC:                             485.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       5645.1622    221.905     25.440      0.000      5191.972  6098.353
x1            -4.9252    429.754     -0.011      0.991      -882.599   872.749
==============================================================================
Omnibus:                        0.028   Durbin-Watson:                   2.079
Prob(Omnibus):                  0.986   Jarque-Bera (JB):                0.228
Skew:                          -0.001   Prob(JB):                        0.892
Kurtosis:                       2.587   Cond. No.                         6.83
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [1]:
fracPass[6]

NameError: name 'fracPass' is not defined

In [22]:
summaryList[3]

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  506
Model:                          Logit   Df Residuals:                      501
Method:                           MLE   Df Model:                            4
Date:                Fri, 27 Jan 2017   Pseudo R-squ.:                  0.1027
Time:                        18:54:27   Log-Likelihood:                -299.07
converged:                       True   LL-Null:                       -333.31
                                        LLR p-value:                 4.737e-14
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         -3.2169      0.560     -5.744      0.000        -4.315    -2.119
x1             0.9575      0.161      5.949      0.000         0.642     1.273
x2             0.2266      0.037      6.132      0.000         0.154     0.299
x3             0.0061      0.008      0.777      0.437        -0.009     0.022
x4            -4.5615      2.489     -1.832      0.067        -9.441     0.318
==============================================================================
"""

In [27]:
for logit in fitResult:
    print logit.params

[-3.21690967  0.95752875  0.22662595  0.00612816 -4.56149964]
[-3.21690967  0.95752875  0.22662595  0.00612816 -4.56149964]
[-3.21690967  0.95752875  0.22662595  0.00612816 -4.56149964]
[-3.21690967  0.95752875  0.22662595  0.00612816 -4.56149964]
[-3.21690967  0.95752875  0.22662595  0.00612816 -4.56149964]
[ -1.57674650e+00   5.45565765e-01   9.05689301e-02   4.09863013e-03
  -4.57760407e+00]
[ -2.43372418e+00   1.06366827e+00   9.06222939e-02   8.66593920e-03
  -2.11586769e+01]
[-2.52409841  0.96942489  0.08641661  0.00895465 -7.74578233]
[ -2.47864903e+00   8.23124108e-01   1.19661970e-01   1.31429987e-02
  -2.21278743e+01]
[ -1.96849596e+00   8.39726856e-01   1.08382420e-01  -3.89530093e-03
  -2.57462965e+01]
[ -1.71670176e+00   6.57409070e-01   1.04211474e-01   2.69006525e-03
  -1.77336375e+01]
[ -2.65447787e+00   8.92519893e-01   1.14171770e-01   1.00854829e-02
  -1.78627769e+01]
[ -2.70357495e+00   7.78729075e-01   1.57010370e-01   5.36427412e-04
  -2.12584322e+01]
[-1.20644752

In [28]:
len(fitResult)

66

In [37]:
len(logitList)

32

In [36]:
len(teams)

32

In [38]:
sigmoids

array([ 0.46167341,  0.50482503,  0.62507412,  0.6655749 ,  0.67358864,
        0.74642147,  0.52399812,  0.69332632,  0.70097383,  0.53417891,
        0.53461365,  0.65262446,  0.73734785,  0.42871473,  0.44904672,
        0.48411366,  0.52790859,  0.41049909,  0.24307017,  0.42690756,
        0.54921685,  0.49014812,  0.54192365,  0.65927166,  0.43193131,
        0.55429316,  0.67078104,  0.40387402,  0.45771872,  0.47647235,
        0.49663667,  0.48202028,  0.51534318,  0.63492121,  0.68427889,
        0.74400082,  0.66406369,  0.72065311,  0.80844722,  0.27118562,
        0.38023762,  0.45614974,  0.5059775 ,  0.52294673,  0.79170166,
        0.67861559,  0.71385564,  0.71002438,  0.80666876,  0.73863809,
        0.82460008,  0.48536506,  0.57361095,  0.48140898,  0.43100128,
        0.42649049,  0.48903698,  0.52082513,  0.51660307,  0.56195941,
        0.67719415,  0.46506518,  0.43382714,  0.40279621,  0.39103634,
        0.23961295,  0.36090997,  0.50269089,  0.4286332 ,  0.44

In [40]:
histogram = pl.hist(sigmoids)

In [42]:
pl.show(histogram)